# 네이버뉴스 본문, 댓글

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

**1. 뉴스 제목, 본문, 날짜, url 수집**

In [3]:
RESULT_PATH = 'C:/Users/snari/python study/5주차/'
now = datetime.now() #파일이름 현 시간으로 저장하기

def get_news(n_url):
    news_detail = []

    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')

    title = bsoup.select('h3#articleTitle')[0].text  
    news_detail.append(title) #제목

    pdate = bsoup.select('.t11')[0].get_text()[:11]
    news_detail.append(pdate) #날짜

    _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip()) #댓글 본문
  
    news_detail.append(n_url) #url
    
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany) #신문사

    return news_detail

def crawler(maxpage,query,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    f = open("C:/Users/snari/python study/5주차/contents_text.txt", 'w', encoding='utf-8')
    
    while page < maxpage_t:
    
        print(page)
    
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        req = requests.get(url)
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
            #print(soup)
    
        for urls in soup.select("._sp_each_url"):
            try :
                #print(urls["href"])
                if urls["href"].startswith("https://news.naver.com"):
                    #print(urls["href"])
                    news_detail = get_news(urls["href"])
                        # pdate, pcompany, title, btext
                    f.write("{}\t{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[4], news_detail[0], news_detail[2],news_detail[3]))  # new style
            except Exception as e:
                print(e)
                continue
        page += 10
    
    
    f.close()
    
def excel_make():
    data = pd.read_csv(RESULT_PATH+'contents_text.txt', sep='\t',header=None, error_bad_lines=False)
    data.columns = ['years','company','title','contents','link']
    print(data)
    
    xlsx_outputFileName = '%s-%s-%s  %s시 %s분 %s초 result.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
    #xlsx_name = 'result' + '.xlsx'
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8')


def main():
    maxpage = input("최대 출력할 페이지수 입력하시오: ") 
    query = input("검색어 입력: ")
    s_date = input("시작날짜 입력(2019.09.19):")  #2019.09.19
    e_date = input("끝날짜 입력(2019.10.19):")   #2019.10.19
    crawler(maxpage,query,s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다. 
    
    excel_make() #엑셀로 만들기 
main()

최대 출력할 페이지수 입력하시오: 2
검색어 입력: 이춘재
시작날짜 입력(2019.09.19):2019.09.19
끝날짜 입력(2019.10.19):2019.09.30
1
https://search.naver.com/search.naver?where=news&query=이춘재&sort=0&ds=2019.09.19&de=2019.09.30&nso=so%3Ar%2Cp%3Afrom20190919to20190930%2Ca%3A&start=1
'NoneType' object has no attribute 'get_text'


b'Skipping line 8: expected 5 fields, saw 10\nSkipping line 9: expected 5 fields, saw 7\nSkipping line 10: expected 5 fields, saw 7\nSkipping line 17: expected 5 fields, saw 11\n'


          years     company  \
0   2019.09.30.        노컷뉴스   
1   2019.09.30.         YTN   
2   2019.09.30.       마이데일리   
3   2019.09.30.       헤럴드경제   
4   2019.09.30.        JTBC   
5   2019.09.30.        JTBC   
6   2019.09.29.         YTN   
7   2019.09.29.         뉴스1   
8   2019.09.29.        세계닷컴   
9   2019.09.29.      edaily   
10  2019.09.29.  MoneyToday   
11  2019.09.29.         채널A   
12  2019.09.28.        JTBC   
13  2019.09.28.        부산일보   

                                                title  \
0                        이춘재 8차 대면조사…4차 사건 DNA 결과는 아직   
1                             이춘재 대면조사 재개...여전히 혐의 부인   
2               '화성연쇄살인사건' 이춘재 다룬 '그알'…2049 주말 시청률 1위   
3   ‘그알’ 화성연쇄살인 2부, 처제성폭행 사건 분석 통한 이춘재의 기이한 발언과 행적 추적   
4                  '화성사건' 발생 2년 전…"이춘재 닮은 20대가 흉기 위협"   
5                     마주친 시기는 이춘재 군 복무 시절…휴가 중 범행 가능성   
6                  화성 연쇄살인 목격자 버스안내원, 이춘재 사진에 "범인 맞다"   
7                      화성살인사건 핵심 목격자 “범인 얼굴, 이춘재와 같다”   
8                    

**2. 댓글 수집**

In [4]:
RESULT_PATH ='C:/Users/snari/python study/5주차/'  #결과 저장할 경로
now = datetime.now()

# 댓글을 달 빈 리스트를 생성합니다.
List=[]
result={}
# 라이브러리를 로드합니다.
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint

# 네이버 뉴스 url을 입력합니다.
url="https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=016&aid=0001591741" #referer 다 들어가서 분석

oid=url.split("oid=")[1].split("&")[0]
aid=url.split("aid=")[1]
page=1    
header = {
    "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
    "referer":url,
    
} 
while True :
    c_url="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE" 
# 파싱하는 단계입니다.
#네트워크에서 댓글 부분 url 직접 가져오기
    re=requests.get(c_url,headers=header)
    cont=BeautifulSoup(r.content,"html.parser")    
    total_comm=str(cont).split('comment":')[1].split(",")[0]
   
    match=re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
# 댓글을 리스트에 중첩합니다.
    List.append(match)
# 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
    if int(total_comm) <= ((page) * 20):
        result = {"comment" : List}
        df = pd.DataFrame(result)
        break
    else : 
        page+=1
    
    
    # 새로 만들 파일이름 지정
#     outputFileName = '%s-%s-%s  %s시 %s분 %s초 merging.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
#     df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')        
        
    

'\n# 여러 리스트들을 하나로 묶어 주는 함수입니다.\ndef flatten(l):\n    flatList = []\n    for elem in l:\n        # if an element of a list is a list\n        # iterate over this list and add elements to flatList \n        if type(elem) == list:\n            for e in elem:\n                flatList.append(e)\n        else:\n            flatList.append(elem)\n    return flatList\n\n\n# 리스트 결과입니다.\nflatten(List)\n'

In [5]:
df

,comment
0,"[""버ㄹ ㅓ지...죄없는 이를 밟고..올라간 버ㄹ ㅓ지.."", ""이런건 국가나 정부..."
1,"[""남의인생 밟고올라서서 살고있는 인간쓰레기들 결국 죗값은 네들이 죽어서라도받는다..."
2,"[""OOO 얼굴을 까발려야지 쓰레기경찰로"", ""그래 너거 자식들 얼마나 잘돼는지 ..."
3,"[""진범은 못잡는 무능력자들이 엄한 사람 인생 망치고 특진에 연금에 다 압수해라"",..."
4,"[""검찰,사법부개혁도 중요하지만 경찰개혁도 중요하다...쓰레기 경찰들도 많았다..""..."
5,"[""이게 나라냐?"", ""살인의 추억 영화에서 하던 ㅂ ㅓ러지 짓들이 진짜였네"", ""..."
6,"[""인간 쓰레기들 지 자식들한테는 바르게 살라고 했겠지"", ""시간이 지났다고 양..."
7,"[""사죄 하세요"", ""하도 기막혀 어이가 없다."", ""공소시효라는건 과거에 컴퓨터도..."
8,"[""ㅅㅂ 말년에는 고통받는 삶을 살긴바란다"", ""당시 화성 연쇄 살인 관련 수사한 ..."
9,"[""공소시효 없애라"", ""ㅋㅋㅋ 여억쉬~~ ㅁㅣㄱㅐ 종자 넘치나는 ㄱㅐ~..."
